[View in Colaboratory](https://colab.research.google.com/github/workyasu/analytics-practice-machine-learning/blob/master/bank.ipynb)

In [0]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

##Google Driveの環境構築

In [0]:
# google driveのファイルを使う時に使用
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Google Driveからデータセットの読み込み

In [45]:
# google check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# google driveのファイル取得
downloaded = drive.CreateFile({'id':'1AIbCQfoWtvDG3ToZ7PADLyWUbuq_QQAK'})

# Download the file to a local disk as 'sample.csv'.
downloaded.GetContentFile('bank-full.csv')


# データの読み込み
bank = pd.read_csv("bank-full.csv", sep=";")
bank.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [0]:
# 特徴量を構築する関数
def makefeature(x):
    # 数値変換のスケーリング
    x = bank.drop('y',1)

    # 特定の列を指定うして処理
    cn_num = ['age','balance','day','duration','campaign','pdays','previous']
    x_num = x[cn_num]
    x[cn_num] = (x_num - x_num.mean())/x_num.std()

    # ダミー変数への変換
    x_dum = pd.get_dummies(x)
    return x_dum

In [0]:
# 特徴量を取得
features, label = makefeature(bank.drop('y',1)), bank.y

## SVMによる予測

In [0]:
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report

In [0]:
# 訓練データとテストデータの作成
random_state = np.random.RandomState(123)
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=.3, random_state=random_state)

In [50]:
# RBFカーネルのSVMによる予測モデル構築
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [51]:
# クラスラベルの予測
pred = clf.predict(x_test)

# クラスごとのPrecision, Recall, F値, Accuracy算出
print(metrics.classification_report(y_test, pred, target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.91      0.98      0.94     11998
        yes       0.65      0.28      0.39      1566

avg / total       0.88      0.90      0.88     13564



## Random Forestによる予測

In [0]:
import numpy as np
from sklearn import ensemble
from sklearn import metrics

In [0]:
# 訓練データとテストデータの作成
random_state = np.random.RandomState(123)
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=.3, random_state=random_state)

In [54]:
# Random Forestによる予測モデル構築
clf = ensemble.RandomForestClassifier(n_estimators=500, random_state=random_state)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False,
            random_state=<mtrand.RandomState object at 0x7f82d0a14168>,
            verbose=0, warm_start=False)

In [55]:
# クラスラベルの予測
pred = clf.predict(x_test)

# クラスごとの適合率,再現率,F値,正解率の算出
print(metrics.classification_report(y_test,pred,target_names=['no','yes']))

             precision    recall  f1-score   support

         no       0.92      0.97      0.95     11998
        yes       0.67      0.39      0.49      1566

avg / total       0.89      0.91      0.90     13564



In [67]:
y_test

7281     yes
19469     no
31637     no
22484     no
35919     no
15644     no
12619     no
30249     no
11489     no
21513     no
10535     no
32198     no
6932      no
44816    yes
22607     no
44365     no
15098     no
15263    yes
32340     no
40989     no
43280     no
14295     no
34445     no
6259      no
2261      no
30975     no
17353     no
16699     no
38618     no
23687     no
        ... 
4275      no
5346      no
25849     no
21619     no
849      yes
42251     no
35507     no
261       no
35072     no
5533      no
40606     no
26538     no
17665     no
43463    yes
6348      no
40175    yes
8132      no
6049      no
3135      no
18001    yes
17453     no
14624     no
365       no
2539      no
35405     no
7839      no
35466     no
5695      no
42570     no
11282     no
Name: y, Length: 13564, dtype: object

## クロスバリデーション

In [0]:
from sklearn import cross_validation as cv
from sklearn import preprocessing

In [57]:
# 層別k分割
skf = cv.StratifiedKFold(y_train, 10)

# 各分割における訓練データ、テストデータの行番号の表示
for train, test in skf:
    print("%s %s" % (train, test))

[ 3084  3098  3120 ... 31644 31645 31646] [   0    1    2 ... 3177 3178 3180]
[    0     1     2 ... 31644 31645 31646] [3084 3098 3120 ... 6367 6368 6369]
[    0     1     2 ... 31644 31645 31646] [6071 6075 6076 ... 9539 9540 9541]
[    0     1     2 ... 31644 31645 31646] [ 9168  9173  9187 ... 12759 12760 12761]
[    0     1     2 ... 31644 31645 31646] [11997 11998 12004 ... 15895 15896 15897]
[    0     1     2 ... 31644 31645 31646] [15222 15231 15238 ... 19059 19062 19063]
[    0     1     2 ... 31644 31645 31646] [18552 18585 18587 ... 22204 22205 22206]
[    0     1     2 ... 31644 31645 31646] [21659 21662 21671 ... 25345 25346 25347]
[    0     1     2 ... 31644 31645 31646] [25106 25138 25143 ... 28502 28503 28504]
[    0     1     2 ... 28502 28503 28504] [28269 28282 28288 ... 31644 31645 31646]


In [0]:
# RBFカーネルのサポートベクターマシン
clf = svm.SVC()
# クラスラベルを1, 0に変換
lb = preprocessing.LabelBinarizer()
y_train_bin = lb.fit_transform(y_train).ravel()

In [59]:
# クロスバリデーションによる評価指標
cv.cross_val_score(clf, x_train, y_train_bin, cv=skf, scoring='f1')

array([0.36329588, 0.38420108, 0.36111111, 0.38185255, 0.40740741,
       0.38931298, 0.39344262, 0.41666667, 0.38888889, 0.38356164])

In [60]:
# クロスバリデーションによる予測結果
pred = cv.cross_val_predict(clf, x_train, y_train, cv=skf)
print(classification_report(y_train, pred, target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.91      0.98      0.94     27924
        yes       0.65      0.28      0.39      3723

avg / total       0.88      0.90      0.88     31647



## Grid Search

In [61]:
from sklearn import grid_search as gs

/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [62]:
# 探索するハイパーパラメータの範囲
param_grid = [
    {'C': [0.5, 1], 'gamma': [0.05, 0.1]}
]

#各ハイパーパラメータに対するGrid Searchの実行
svc = svm.SVC()
clf = gs.GridSearchCV(svc, param_grid, cv=10)
clf.fit(x_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [0.5, 1], 'gamma': [0.05, 0.1]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [63]:
pred = clf.predict(x_test)
print(classification_report(y_test, pred, target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.92      0.98      0.95     11998
        yes       0.66      0.35      0.46      1566

avg / total       0.89      0.90      0.89     13564

